In [30]:
from elasticsearch import Elasticsearch, helpers
from collections import defaultdict
from dateutil.parser import parse

In [4]:
es = Elasticsearch(
    "http://127.0.0.1:9200",
    verify_certs=False,
    basic_auth=("elastic", "123456"),
)
idx = "docs"

In [10]:
es.indices.get_mapping(index=idx).body

{'docs': {'mappings': {'properties': {'borad': {'type': 'keyword'},
    'comments': {'type': 'nested',
     'properties': {'content': {'type': 'text',
       'analyzer': 'ik_max_word_plus',
       'search_analyzer': 'ik_smart_plus'},
      'content_aste': {'type': 'nested',
       'properties': {'a': {'type': 'keyword'},
        'o': {'type': 'keyword'},
        'p': {'type': 'float'}}},
      'date': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss.SSS'},
      'username': {'type': 'text',
       'analyzer': 'ik_max_word_plus',
       'search_analyzer': 'ik_smart_plus'}}},
    'context': {'type': 'text',
     'analyzer': 'ik_max_word_plus',
     'search_analyzer': 'ik_smart_plus'},
    'context_aste': {'type': 'nested',
     'properties': {'a': {'type': 'keyword'},
      'o': {'type': 'keyword'},
      'p': {'type': 'keyword'}}},
    'context_vector': {'type': 'dense_vector',
     'dims': 1792,
     'index': True,
     'similarity': 'cosine',
     'index_options': {'type': 'int8_hnsw',

In [20]:
body = {
    "_source": False,
    "query": {
        "match_all": {},
    },
    "fields": ["link", "crawl_time"],
}
res = list(helpers.scan(es, query=body, index=idx))

In [32]:
dd = defaultdict(list)

In [33]:
for i in res:
    dd[i["fields"]["link"][0]].append((i["_id"], parse(i["fields"]["crawl_time"][0])))

In [42]:
del_ids_list = []
for k, v in dd.items():
    if len(v) > 1:
        tmp = list(zip(*v))
        argmax = tmp[1].index(max(tmp[1]))
        del_ids = list(tmp[0])
        del_ids.pop(argmax)
        del_ids_list.extend(del_ids)

In [45]:
def gen_del_body(del_ids_list):
    for i in del_ids_list:
        yield {
            "_op_type": "delete",
            "_index": idx,
            "_id": i,
        }

In [46]:
helpers.bulk(es, gen_del_body(del_ids_list))

(5254, [])